In [19]:
from dotenv import load_dotenv
import os

from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

In [11]:
load_dotenv(override=True)

openai_model = "gpt-4o-mini"
openai_llm = ChatOpenAI(model=openai_model, temperature=0.1)

groq_model = "gemma2-9b-it"
groq_llm = ChatGroq(model=groq_model)

## Basic Invocation

In [12]:
groq_llm.invoke("Hi GPT")

AIMessage(content='Hi! 👋  What can I do for you today? 😄\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 11, 'total_tokens': 27, 'completion_time': 0.029090909, 'prompt_time': 0.001180179, 'queue_time': 0.25046044, 'total_time': 0.030271088}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--660d34c9-7162-481c-8a5c-c8bbb1626fdf-0', usage_metadata={'input_tokens': 11, 'output_tokens': 16, 'total_tokens': 27})

## Passing prompts directly via tuples

In [ ]:
system_prompt = "You are a helpful assistant who is able to provide answers to questions"
user_prompt = "Answer the following question: What are the names of planet in Earth's solar system."

basic_prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    ("user", "{user_prompt}")
])

# print(basic_prompt)

chain = basic_prompt | groq_llm
chain.invoke({"system_prompt": system_prompt, "user_prompt":user_prompt})

AIMessage(content="Here are the planets in Earth's solar system, in order from the sun:\n\n- Mercury\n- Venus\n- Earth\n- Mars\n- Jupiter\n- Saturn\n- Uranus\n- Neptune \n\n\nLet me know if you have any other questions!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 44, 'total_tokens': 100, 'completion_time': 0.101818182, 'prompt_time': 0.001647019, 'queue_time': 0.25350609, 'total_time': 0.103465201}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--70798e80-a776-4415-ac53-a7f79bc07ed0-0', usage_metadata={'input_tokens': 44, 'output_tokens': 56, 'total_tokens': 100})

### Add OutputParser

In [18]:
system_prompt = "You are a helpful assistant who is able to provide answers to questions"
user_prompt = "Answer the following question: What are the names of planet in Earth's solar system."

basic_prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    ("user", "{user_prompt}")
])

# print(basic_prompt)

chain = basic_prompt | groq_llm | StrOutputParser()
chain.invoke({"system_prompt": system_prompt, "user_prompt":user_prompt})

"Here are the names of the planets in Earth's solar system, in order from the Sun:\n\n1. **Mercury**\n2. **Venus**\n3. **Earth**\n4. **Mars**\n5. **Jupiter**\n6. **Saturn**\n7. **Uranus**\n8. **Neptune** \n\n\nLet me know if you have any other questions! 🪐🚀\n"

## Use a JsonOutputParser

## Use SystemMessagePromptTemplate and HumanMessagePromptTemplate

In [ ]:
system_message = SystemMessagePromptTemplate.from_template("""
You are a helpful assistant that helps generate article titles.
""")
                                                           
article_text = """AI is a useful tool for creative purposes. It can write text, create images and videos and act as a useful
assistant to the creator. Rather than thinking of AI as a threat, we need to embrace its power to be more productive
and experiment more and delegate the mundaneness to our digital assistant."""

In [6]:
user_prompt = HumanMessagePromptTemplate.from_template(
    """You are looking at an article text:
    ---
    {article_text}
    --- 
    Generate a suitable title for the same.
    The tone of the title should be comic yet catchy.""",
    input_variables=["article_text"]
)

In [7]:
# Using input_variables in the prompt template
user_prompt.format(article_text=article_text)

HumanMessage(content='You are looking at an article text:\n    ---\n    AI is a useful tool for creative purposes. It can write text, create images and videos and act as a useful\nassistant to the creator. Rather than thinking of AI as a threat, we need to embrace its power to be more productive\nand experiment more and delegate the mundaneness to our digital assistant.\n    --- \n    Generate a suitable title for the same.\n    The tone of the title should be comic yet catchy.', additional_kwargs={}, response_metadata={})

In [8]:
first_prompt = ChatPromptTemplate.from_messages([system_message, user_prompt])

In [9]:
print(first_prompt.format(article_text=article_text))

System: 
You are a helpful assistant that helps generate article titles.

Human: You are looking at an article text:
    ---
    AI is a useful tool for creative purposes. It can write text, create images and videos and act as a useful
assistant to the creator. Rather than thinking of AI as a threat, we need to embrace its power to be more productive
and experiment more and delegate the mundaneness to our digital assistant.
    --- 
    Generate a suitable title for the same.
    The tone of the title should be comic yet catchy.


Now my prompt is ready, I have defined my model as well as input. Now its time to feed the prompt to the model.

# Create chain using prompt and LLM.

The chain will do the following:

1. Format the prompt
2. generate using LLM
3. Output


Chain is created using LCEL.

In the following chain the output of LLM is an AIMessage

In [10]:
chain_one = ({"article_text": lambda x: x["article_text"]} 
             | first_prompt
             | llm
             | {"article_title": lambda x: x.content})

In [11]:
title_output = chain_one.invoke({"article_text": article_text})["article_title"]

In [12]:
title_output

'"Unleash Your Inner Picasso: How AI Can Turn You into a Creative Superhero!"'

* So we have generated article title using LLM.

* Next we'll patch this title with the article and generate an article summary.

In [13]:
second_system_prompt = SystemMessagePromptTemplate.from_template("""You are a helpful assistant who is able to help in building good articles.
                                                                 """)


second_user_prompt = HumanMessagePromptTemplate.from_template("""
You have access to an article: 
{article_text} alongwith its title: {article_title}. 
Generate a crisp summary for the article using this information. Make sure that the summary is less than 100 characters.
The description should be in a SEO compatible format and suitable to show up on Google when this appears in search results""",
input_variables = ["article_text", "article_title"])


second_prompt = ChatPromptTemplate([second_system_prompt, second_user_prompt])

In [14]:
chain_two = (
    {
        "article_text": lambda x: x["article_text"],
        "article_title": lambda x: x["title"]
    }
    | second_prompt
    | llm
    | {"article_summary": lambda x: x.content}
)

In [15]:
article_summary = chain_two.invoke({"article_text": article_text, "title": title_output })
article_summary

{'article_summary': 'Discover how AI enhances creativity, empowering you to produce art and delegate mundane tasks.'}

* We'll ask the LLM for unstructured outputs helping the user to write better. 

In [16]:
third_user_prompt = HumanMessagePromptTemplate.from_template("""
You are tasked with writing a new paragraph for the article.
The article is as follows:
{article_text}
You can pick some part of the text and provide constructive feedback to the
user so that they can improve their own writing.                                                  
""",
input_variables = ["article_text"])

In [17]:
third_prompt = ChatPromptTemplate.from_messages([system_message, third_user_prompt])

In [18]:
from pydantic import BaseModel, Field

In [19]:
class Paragraph(BaseModel):
    original_paragraph: str = Field(description="The original paragraph")
    edited_paragraph: str = Field(description="The improved paragraph as edited by the model")
    feedback: str = Field(description="Constructive feedback on the original paragraph")

In [20]:
structured_llm = llm.with_structured_output(Paragraph)

In [21]:
chain_three = (
    {"article_text": lambda x: x["article_text"]}
    | third_prompt
    | structured_llm
    | {
        "original_paragraph": lambda x: x.original_paragraph,
        "edited_paragraph": lambda x: x.edited_paragraph,
        "feedback": lambda x: x.feedback
    }
)

In [22]:
third_output = chain_three.invoke({"article_text": article_text})

In [23]:
third_output

{'original_paragraph': 'AI is a useful tool for creative purposes. It can write text, create images and videos and act as a useful to the creator. Rather than thinking of AI as a threat, we need to embrace its power to be more productive and experiment more and delegate the mundaneness to our digital assistant.',
 'edited_paragraph': 'AI serves as a powerful ally in the realm of creativity, capable of generating text, crafting images, and producing videos. Instead of viewing AI as a potential threat, we should harness its capabilities to enhance our productivity, explore new ideas, and offload repetitive tasks to our digital assistants.',
 'feedback': "The original paragraph has a good foundation but could benefit from improved clarity and flow. The phrase 'act as a useful to the creator' is incomplete and should be revised for coherence. Additionally, the sentence structure can be varied to enhance readability. The edited version addresses these issues by providing a more fluid and en